In [6]:
#import libraries
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
#import chart_studio.plotly as py
#import plotly.offline as pyoff
#import plotly.graph_objs as go
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,date
import calendar
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from lifetimes.datasets import load_cdnow_summary_data_with_monetary_value
import xgboost as xgb
from sklearn.metrics import classification_report
from lifetimes.utils  import summary_data_from_transaction_data
from sklearn.cluster import KMeans
import datetime as dt
from sklearn.preprocessing import StandardScaler

In [5]:
!pip install Lifetimes

  Stored in directory: C:\Users\admin\AppData\Local\pip\Cache\wheels\42\62\66\1121afe23ff96af4e452e0d15e68761e3f605952ee075ca99f
Successfully built autograd


In [7]:
#initate plotly
pyoff.init_notebook_mode()

NameError: name 'pyoff' is not defined

In [ ]:
df = pd.read_csv('C:/Users/Administrator/Desktop/FirstChoice/Blended_Data_Features_Removed.csv', low_memory=False)
df['Invoice Date']= pd.to_datetime(df['Invoice Date']) 

In [ ]:
df['InvoiceDay'] = df['Invoice Date'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [ ]:
# print the time period
print('Min : {}, Max : {}'.format(min(df['InvoiceDay']), max(df['InvoiceDay'])))

In [ ]:
# pin the last date
pin_date = max(df['Invoice Date']) + dt.timedelta(1)
print(pin_date)

In [ ]:
# calculate RFM values
rfm = df.groupby('Customer No.').agg({
    'Invoice Date' : lambda x: (pin_date - x.max()).days,
    'Job Card No' : 'count', 
    'Total Amt Wtd Tax.' : 'sum'})

In [ ]:
# rename the columns
rfm.rename(columns = {'Invoice Date' : 'Recency', 
                      'Job Card No' : 'Frequency', 
                      'Total Amt Wtd Tax.' : 'Monetary'}, inplace = True)

In [ ]:
rfm.head()

In [ ]:
# Plot RFM distributions
plt.figure(figsize=(12,10))
# Plot distribution of R
plt.subplot(3, 1, 1); sns.distplot(rfm['Recency'])
# Plot distribution of F
plt.subplot(3, 1, 2); sns.distplot(rfm['Frequency'])
# Plot distribution of M
plt.subplot(3, 1, 3); sns.distplot(rfm['Monetary'])
# Show the plot
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Recency']])
rfm['R'] = kmeans.predict(rfm[['Recency']])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Frequency']])
rfm['F'] = kmeans.predict(rfm[['Frequency']])

In [3]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Monetary']])
rfm['M'] = kmeans.predict(rfm[['Monetary']])

NameError: name 'rfm' is not defined

In [ ]:
rfm.head()

In [ ]:
# Concat RFM KMeans Cluster values to create RFM Segments
def join_rfm(x): return str(x['R'])+str(x['F'])+str(x['M'])
rfm['RFM_Segment_Concat'] = rfm.apply(join_rfm, axis=1)
rfm.head()

In [ ]:
# Count num of unique segments
rfm_count_unique = rfm.groupby('RFM_Segment_Concat')['RFM_Segment_Concat'].nunique()
print(rfm_count_unique.sum())

In [ ]:
# Calculate RFM_Score
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)
print(rfm['RFM_Score'].head())

In [ ]:
rfm.head()

In [ ]:
rfm.RFM_Score.value_counts(sort=True, ascending=False)

In [ ]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 6:
        return 'High Segment'
    elif ((df['RFM_Score'] >= 3) and (df['RFM_Score'] < 6)):
        return 'Mid Segment'
    else:
        return 'Low Segment'

# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console
rfm.head()

In [ ]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = rfm.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)
# Print the aggregated dataset
print(rfm_level_agg)

In [ ]:
import squarify
# rfm_level_agg.columns = rfm_level_agg.columns.droplevel()
rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']
#Create our plot and resize it.
fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(16, 9)
squarify.plot(sizes=rfm_level_agg['Count'], 
              label=['High Segment',
                     'Mid Segment',
                     'Low Segment'], alpha=.6 )
plt.title("RFM Segments",fontsize=55,fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
rfm.head()

In [ ]:
rfm.corr()

In [ ]:
rfm.reset_index(inplace=True)


In [ ]:
codes = {'High Segment':0, 'Mid Segment':1, 'Low Segment':2}
rfm['RFM_Level'] = rfm['RFM_Level'].map(codes)

In [ ]:
rfm.head()

rfm.head()

In [ ]:
rfm.drop(columns=['R','F','M','RFM_Segment_Concat'], inplace=True)

In [ ]:
temp = rfm

In [ ]:
temp.drop(columns=['RFM_Score'], inplace=True)
temp.corr()

In [ ]:
temp.info()

In [ ]:
# Predictors
X = temp.iloc[:,:-1]
# Target
y = temp.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
auc = roc_auc_score(y_val, y_pred)

In [ ]:
y_pred

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier ,GradientBoostingClassifier
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

def run_model(predictors,target, model):
    '''
    Performs model training and tests using ROC-AUC 
    returns AUC score
    '''
    x_train,x_val,y_train,y_val = train_test_split(predictors,target,test_size=0.2,random_state=42)
    model.fit(x_train, y_train)
    y_scores = model.predict(x_val)
    auc = roc_auc_score(y_val, y_scores)
    print('Classification Report:')
    print(classification_report(y_val,y_scores))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_scores)
    print('ROC_AUC_SCORE is',roc_auc_score(y_val, y_scores))
    
    #fpr, tpr, _ = roc_curve(y_test, predictions[:,1])
    
    plt.plot(false_positive_rate, true_positive_rate)
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC curve')
    plt.show()
    return auc

# Predictors
X = temp.iloc[:,:-1]

# Target
y = temp.iloc[:,-1]

# Choosing the models. If you want to specify additional models, kindly specify them as a key-value pair as shown below.
models = {'Logistic Regression':LogisticRegression,'Decision Tree':DecisionTreeClassifier,'Random Forest': RandomForestClassifier,'XGBoost':XGBClassifier,'Gradient Boosting':GradientBoostingClassifier}

for i in models.items():
    # run model
    model = i[1]()
    print('~~~~~~Running for model '+str(model)+'~~~~~~')
    auc = run_model(X, y, model) # train and returns AUC test score
    print('AUC Score = %.2f' %(auc*100) +' %\nOn Model - \n'+str(i[0]))
    print('===='*20)

NameError: name 'temp' is not defined

In [ ]:
df.rename(columns={'Invoice Date':'InvoiceDate'},inplace=True)
df.rename(columns={'Total Amt Wtd Tax.':'Revenue'},inplace=True)

In [ ]:
summary=summary_data_from_transaction_data(df,'Customer No.','InvoiceDate',monetary_value_col='Revenue',observation_period_end='2016')

In [ ]:
from lifetimes import BetaGeoFitter
bgf=BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'],summary['recency'],summary['T'])
print(bgf)

In [ ]:
t=10

summary['pred_num']=round(bgf.conditional_expected_number_of_purchases_up_to_time(t,summary['frequency'],summary['recency'], summary['T']),2)
summary.sort_values(by='pred_num', ascending=False).head(10)

In [ ]:
from lifetimes.plotting import plot_period_transactions
plot_period_transactions(bgf)

In [ ]:
summary[['monetary_value', 'frequency']].corr()

In [ ]:
#Shortlist customers who had at least one repeat purchase with the company. 
shortlisted_customers = summary[summary['frequency']>0]
print(shortlisted_customers.head().reset_index())
print("-----------------------------------------")
print("The Number of Returning Customers are: ",len(shortlisted_customers))

In [ ]:
shortlisted_customers = shortlisted_customers[shortlisted_customers['monetary_value']>0]
shortlisted_customers

In [ ]:
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(shortlisted_customers['frequency'],
        shortlisted_customers['monetary_value'])
print(ggf)

In [ ]:
#After applying Gamma-Gamma model, now we can estimate average transaction value for each customer. 
print(ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']
    ).head(10))

In [ ]:
summary['pred_txn_value'] = round(ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']), 2)
summary.reset_index().head()

In [ ]:
#Calculate Customer Lifetime Value
summary['CLV'] = round(ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    summary['frequency'],
    summary['recency'],
    summary['T'],
    summary['monetary_value'],
    time=12, # months
    discount_rate=0.01 # monthly discount rate ~ 12.7% annually
), 2)

summary.drop(summary.iloc[:, 0:6], inplace=True, axis=1)

In [ ]:
summary.sort_values(by='CLV', ascending=False).head(10).reset_index()

In [ ]:
clv=pd.merge(rfm,summary,on='Customer No.',how='inner')

In [ ]:
clv['no']=np.arange(len(clv))

In [ ]:
customer=clv[['Customer No.','no']]

In [ ]:
clv.drop('Customer No.',axis=1,inplace=True)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x=clv.drop('RFM_Level',axis=1)
y=clv[['RFM_Level']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=52)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [ ]:

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='weighted'))
    

In [ ]:
final=pd.merge(clv,customer,on='no',how='inner')

In [ ]:
#initate plotly
pyoff.init_notebook_mode()

In [ ]:
df = pd.read_csv('C:/Users/Administrator/Desktop/FirstChoice/Blended_Data_Features_Removed.csv', low_memory=False)
df['Invoice Date']= pd.to_datetime(df['Invoice Date']) 

In [ ]:
df['InvoiceDay'] = df['Invoice Date'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [ ]:
# print the time period
print('Min : {}, Max : {}'.format(min(df['InvoiceDay']), max(df['InvoiceDay'])))

In [ ]:
# pin the last date
pin_date = max(df['Invoice Date']) + dt.timedelta(1)
print(pin_date)

In [ ]:
# calculate RFM values
rfm = df.groupby('Customer No.').agg({
    'Invoice Date' : lambda x: (pin_date - x.max()).days,
    'Job Card No' : 'count', 
    'Total Amt Wtd Tax.' : 'sum'})

In [ ]:
# rename the columns
rfm.rename(columns = {'Invoice Date' : 'Recency', 
                      'Job Card No' : 'Frequency', 
                      'Total Amt Wtd Tax.' : 'Monetary'}, inplace = True)

In [ ]:
rfm.head()

In [ ]:
# Plot RFM distributions
plt.figure(figsize=(12,10))
# Plot distribution of R
plt.subplot(3, 1, 1); sns.distplot(rfm['Recency'])
# Plot distribution of F
plt.subplot(3, 1, 2); sns.distplot(rfm['Frequency'])
# Plot distribution of M
plt.subplot(3, 1, 3); sns.distplot(rfm['Monetary'])
# Show the plot
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Recency']])
rfm['R'] = kmeans.predict(rfm[['Recency']])kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Frequency']])
rfm['F'] = kmeans.predict(rfm[['Frequency']])

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm[['Monetary']])
rfm['M'] = kmeans.predict(rfm[['Monetary']])

In [ ]:
rfm.head()

In [ ]:
# Concat RFM KMeans Cluster values to create RFM Segments
def join_rfm(x): return str(x['R'])+str(x['F'])+str(x['M'])
rfm['RFM_Segment_Concat'] = rfm.apply(join_rfm, axis=1)
rfm.head()

In [ ]:
# Count num of unique segments
rfm_count_unique = rfm.groupby('RFM_Segment_Concat')['RFM_Segment_Concat'].nunique()
print(rfm_count_unique.sum())

In [ ]:
# Calculate RFM_Score
rfm['RFM_Score'] = rfm[['R','F','M']].sum(axis=1)
print(rfm['RFM_Score'].head())

In [ ]:
rfm.head()

In [ ]:
rfm.RFM_Score.value_counts(sort=True, ascending=False)

In [ ]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 6:
        return 'High Segment'
    elif ((df['RFM_Score'] >= 3) and (df['RFM_Score'] < 6)):
        return 'Mid Segment'
    else:
        return 'Low Segment'

# Create a new variable RFM_Level
rfm['RFM_Level'] = rfm.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console
rfm.head()

In [ ]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = rfm.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)
# Print the aggregated dataset
print(rfm_level_agg)

In [ ]:
import squarify
# rfm_level_agg.columns = rfm_level_agg.columns.droplevel()
rfm_level_agg.columns = ['RecencyMean','FrequencyMean','MonetaryMean', 'Count']
#Create our plot and resize it.
fig = plt.gcf()
ax = fig.add_subplot()
fig.set_size_inches(16, 9)
squarify.plot(sizes=rfm_level_agg['Count'], 
              label=['High Segment',
                     'Mid Segment',
                     'Low Segment'], alpha=.6 )
plt.title("RFM Segments",fontsize=55,fontweight="bold")
plt.axis('off')
plt.show()

In [ ]:
rfm.head()

In [ ]:
rfm.corr()

In [ ]:
rfm.reset_index(inplace=True)


In [ ]:
codes = {'High Segment':0, 'Mid Segment':1, 'Low Segment':2}
rfm['RFM_Level'] = rfm['RFM_Level'].map(codes)

In [ ]:
rfm.head()

In [ ]:
rfm.drop(columns=['R','F','M','RFM_Segment_Concat'], inplace=True)

In [ ]:
temp = rfm

In [ ]:
temp.drop(columns=['RFM_Score'], inplace=True)
temp.corr()

In [ ]:
temp.info()

In [ ]:
# Predictors
X = temp.iloc[:,:-1]
# Target
y = temp.iloc[:,-1]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
auc = roc_auc_score(y_val, y_pred)

In [ ]:
y_pred

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier ,GradientBoostingClassifier
from xgboost import XGBClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import roc_auc_score ,mean_squared_error,accuracy_score,classification_report,roc_curve,confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

def run_model(predictors,target, model):
    '''
    Performs model training and tests using ROC-AUC 
    returns AUC score
    '''
    x_train,x_val,y_train,y_val = train_test_split(predictors,target,test_size=0.2,random_state=42)
    model.fit(x_train, y_train)
    y_scores = model.predict(x_val)
    auc = roc_auc_score(y_val, y_scores)
    print('Classification Report:')
    print(classification_report(y_val,y_scores))
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_scores)
    print('ROC_AUC_SCORE is',roc_auc_score(y_val, y_scores))
    
    #fpr, tpr, _ = roc_curve(y_test, predictions[:,1])
    
    plt.plot(false_positive_rate, true_positive_rate)
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC curve')
    plt.show()
    return auc

# Predictors
X = temp.iloc[:,:-1]

# Target
y = temp.iloc[:,-1]

# Choosing the models. If you want to specify additional models, kindly specify them as a key-value pair as shown below.
models = {'Logistic Regression':LogisticRegression,'Decision Tree':DecisionTreeClassifier,'Random Forest': RandomForestClassifier,'XGBoost':XGBClassifier,'Gradient Boosting':GradientBoostingClassifier}

for i in models.items():
    # run model
    model = i[1]()
    print('~~~~~~Running for model '+str(model)+'~~~~~~')
    auc = run_model(X, y, model) # train and returns AUC test score
    print('AUC Score = %.2f' %(auc*100) +' %\nOn Model - \n'+str(i[0]))
    print('===='*20)

In [ ]:
df.rename(columns={'Invoice Date':'InvoiceDate'},inplace=True)
df.rename(columns={'Total Amt Wtd Tax.':'Revenue'},inplace=True)

In [ ]:
summary=summary_data_from_transaction_data(df,'Customer No.','InvoiceDate',monetary_value_col='Revenue',observation_period_end='2016')

In [ ]:
from lifetimes import BetaGeoFitter
bgf=BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'],summary['recency'],summary['T'])
print(bgf)

In [ ]:
t=10

summary['pred_num']=round(bgf.conditional_expected_number_of_purchases_up_to_time(t,summary['frequency'],summary['recency'], summary['T']),2)
summary.sort_values(by='pred_num', ascending=False).head(10)

In [ ]:
from lifetimes.plotting import plot_period_transactions
plot_period_transactions(bgf)

In [ ]:
summary[['monetary_value', 'frequency']].corr()

In [ ]:
#Shortlist customers who had at least one repeat purchase with the company. 
shortlisted_customers = summary[summary['frequency']>0]
print(shortlisted_customers.head().reset_index())
print("-----------------------------------------")
print("The Number of Returning Customers are: ",len(shortlisted_customers))

In [ ]:
shortlisted_customers = shortlisted_customers[shortlisted_customers['monetary_value']>0]
shortlisted_customers

In [ ]:
from lifetimes import GammaGammaFitter
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(shortlisted_customers['frequency'],
        shortlisted_customers['monetary_value'])
print(ggf)

In [ ]:
#After applying Gamma-Gamma model, now we can estimate average transaction value for each customer. 
print(ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']
    ).head(10))

In [ ]:
summary['pred_txn_value'] = round(ggf.conditional_expected_average_profit(
        summary['frequency'],
        summary['monetary_value']), 2)
summary.reset_index().head()

In [ ]:
#Calculate Customer Lifetime Value
summary['CLV'] = round(ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    summary['frequency'],
    summary['recency'],
    summary['T'],
    summary['monetary_value'],
    time=12, # months
    discount_rate=0.01 # monthly discount rate ~ 12.7% annually
), 2)

summary.drop(summary.iloc[:, 0:6], inplace=True, axis=1)

In [ ]:
summary.sort_values(by='CLV', ascending=False).head(10).reset_index()

In [ ]:
clv=pd.merge(rfm,summary,on='Customer No.',how='inner')

In [ ]:
clv['no']=np.arange(len(clv))

In [ ]:
customer=clv[['Customer No.','no']]

In [ ]:
clv.drop('Customer No.',axis=1,inplace=True)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
x=clv.drop('RFM_Level',axis=1)
y=clv[['RFM_Level']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=52)

In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [ ]:

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='weighted'))
    

In [ ]:
final=pd.merge(clv,customer,on='no',how='inner')

# CLV DONE FILE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime,date
import calendar
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from lifetimes.datasets import load_cdnow_summary_data_with_monetary_value
import xgboost as xgb
from sklearn.metrics import classification_report
from lifetimes.utils  import summary_data_from_transaction_data

In [ ]:
customer=pd.read_csv('C:/Users/Administrator/Desktop/FirstChoice/Customer1.csv')
invoice=pd.read_csv('C:/Users/Administrator/Desktop/FirstChoice/Final_invoice.csv')


In [ ]:
invoice.rename(columns={'Custor No.':'Customer No.'},inplace=True)
invoice['Customer No.']=invoice['Customer No.'].apply(lambda x:x.strip())
customer['Customer No.']=customer['Customer No.'].apply(lambda x:str(x))

In [ ]:
join=pd.merge(customer[['Customer No.','Business Partner','Occuption']],invoice[['Area / Locality','CITY','Claim No.','Cust Type','Customer No.','District','Insurance Company','Invoice Date','Job Card No','JobCard Date','KMs Reading','Make','Model','Pin code','ODN No.','Order Type','Plant Name1','Regn No','Total Amt Wtd Tax.']],how='inner', on ='Customer No.')

In [ ]:
join.isnull().sum()
join['occuption']=join['occuption'].apply(lambda x:'Others' if x=='0' else x )

In [ ]:
join['District']=join['District'].apply(lambda x:str(x))
join['District']=join['District'].apply(lambda x:x.lower())
city=join['District'].unique()

In [ ]:
join['Model']=join['Model'].fillna(value='Others')
join['Insurance Company']=join['Insurance Company'].fillna(value='Others')
#join.isnull().sum()

In [ ]:
join['Invoice Date']=join['Invoice Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date()) 
join['monthno']=join['Invoice Date'].apply(lambda x:x.month)

In [ ]:
join['month']=join['monthno'].apply(lambda x:calendar.month_name[x])
join.rename(columns={'Invoice Date':'InvoiceDate'},inplace=True)
join.rename(columns={'Total Amt Wtd Tax.':'Revenue'},inplace=True)

In [ ]:
summary=summary_data_from_transaction_data(join,'Customer No.','InvoiceDate',monetary_value_col='Revenue',observation_period_end='2016')

In [ ]:
summary['frequency'].plot(kind='hist',bins=50)
print(summary['frequency'].describe())
one_time_buyer=round(sum(summary['frequency']==0)/float(len(summary))*(100),2)
print('one time buyer: ',one_time_buyer)

In [ ]:
from lifetimes import BetaGeoFitter
bgf=BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(summary['frequency'],summary['recency'],summary['T'])
print(bgf)

In [ ]:
bgf.summary

In [ ]:
t=10

summary['pred_num']=round(bgf.conditional_expected_number_of_purchases_up_to_time(t,summary['frequency'],summary['recency'], summary['T']),2)
summary.sort_values(by='pred_num', ascending=False).head(10)

In [ ]:
from lifetimes.plotting import plot_period_transactions
plot_period_transactions(bgf)

In [ ]:

t = 500
individual = summary.loc[1]
bgf.predict(t, individual['frequency'], individual['recency'], individual['T'])

In [ ]:
summary[['monetary_value', 'frequency']].corr()

In [ ]:
#Shortlist customers who had at least one repeat purchase with the company. 
shortlisted_customers = summary[summary['frequency']>0]
print(shortlisted_customers.head().reset_index())
print("-----------------------------------------")
print("The Number of Returning Customers are: ",len(shortlisted_customers))

In [ ]:
shortlisted_customers = shortlisted_customers[shortlisted_customers['monetary_value']>0]
shortlisted_customers